In [1]:
import pandas as pd

In [44]:
# districts 

df = pd.read_csv('./outlier_data.csv')

In [3]:
districts = df['districtname'].sort_values().unique()

In [4]:
districts = pd.DataFrame(districts)

In [5]:
districts = districts.reset_index()

In [6]:
districts.columns = ["id", "name"]

In [7]:
#from sqlalchemy import create_engine

In [8]:
#engine = create_engine("sqlite:///db.sqlite")

In [9]:
#districts.to_sql("district", engine, if_exists='append', index=False)

In [10]:
pop = pd.read_csv("./pop.csv")

In [11]:
cols = list(pop.columns)

In [12]:
cols.pop(cols.index("id"))

'id'

In [13]:
cols

['year',
 'male',
 'female',
 'total',
 'childbearing_age',
 'pregnants',
 'not_pregnant',
 'births',
 'u1',
 'u5',
 'u15',
 'suspect_tb']

In [14]:
pop = pd.merge(pop, districts, left_on="id", right_on="name", suffixes=("", "dis"))

In [15]:
pop = pop[cols+["iddis"]][["iddis"]+cols]

In [16]:
pop = pop.rename(columns={"iddis": "district_id", "pregnants": "pregnant"})

In [17]:
#pop.to_sql("population", engine, if_exists="append", index=False)

In [18]:
facility = df[["districtname", "orgUnit", "facility_name"]]

In [19]:
facility = pd.merge(facility, districts, left_on="districtname", right_on="name")[["id", "orgUnit", "facility_name"]]

In [20]:
facility.columns = ["district_id", "id", "name"]

In [21]:
facility = facility[["id", "name", "district_id"]].drop_duplicates().reset_index(drop=True)

In [22]:
#facility.to_sql("facility", engine, if_exists='append', index=False)

In [23]:
indicator = pd.read_csv("./columns.csv")

In [24]:
indicator = indicator[indicator.index>5].reset_index(drop=True)

In [25]:
indicator.columns = ["id", "name", "view"]

In [26]:
#indicator.to_sql("indicator", engine, index=False, if_exists="append")

In [27]:
indicator_groups = pd.read_csv("./groups.csv")[["Choose an indicator group", "Choose an indicator"]]

In [28]:
indicator_groups.columns = ["group", "name"]

In [29]:
indicator_groups = pd.merge(indicator_groups, indicator)[["group", "id"]]

In [30]:
indicator_groups.columns = ["name", "indicator_id"]

In [31]:
#indicator_groups.to_sql("indicator_group", engine, if_exists="append", index=False)

In [32]:
# Create dropdown view

dropdown_indicator = '''
CREATE VIEW dropdown_indicator AS 
SELECT indicator_group.name as 'Choose a group',
       indicator.name as 'Choose an indicator'
FROM indicator 
JOIN indicator_group
WHERE indicator.id = indicator_group.indicator_id
'''

In [33]:
#from sqlalchemy.orm import sessionmaker

In [34]:
#Session = sessionmaker(engine)

In [35]:
#session = Session()

In [36]:
#session.execute(dropdown_indicator)

In [37]:
def get_tall_from_wide(df, value_column="value"):
    df = df.drop(columns=["districtname", "year", "month", "actual_105_1_reporting", "type", "facility_name", "expected_105_1_reporting"])
    df = df.melt(id_vars=["orgUnit", "date"])
    df = df[~pd.isna(df.value)]
    df = df.sort_values(by=["date", "orgUnit"])
    df.columns = ["facility_id", "date", "indicator", value_column]
    df = df.reset_index(drop=True)
    return df

In [45]:
df = get_tall_from_wide(df, "value_raw")

In [39]:
df_std = pd.read_csv('./std_no_outlier_data.csv')

In [40]:
df_std = get_tall_from_wide(df_std, "value_std")

In [46]:
df = pd.merge(df, df_std)

In [47]:
del df_std

In [48]:
df_iqr = pd.read_csv('./iqr_no_outlier_data.csv')

In [49]:
df_iqr = get_tall_from_wide(df_iqr, "value_iqr")

In [50]:
df = pd.merge(df, df_iqr)

In [51]:
del df_iqr

In [ ]:
#df.to_sql("repository", engine, if_exists="append", index=False)

In [ ]:
#el df

In [ ]:
dataset_query = '''
CREATE VIEW dataset AS
SELECT district_name, facility_name, date, name as indicator_name, value_raw, value_std, value_iqr, value_rep
FROM (
      SELECT *
      FROM repository
      JOIN indicator
      WHERE repository.indicator = indicator.id
      )
JOIN
    (
    SELECT facility.id, facility.name as facility_name, district.name as district_name
    FROM facility
    JOIN district
    WHERE district.id = facility.district_id
    ) as facilities
    WHERE facility_id = facilities.id;
'''

In [ ]:
# dropdown view directly in the database

In [52]:
df

,facility_id,date,indicator,value,value_std,value_iqr
0,A0KeWapELXA,2018-01-01,opd_attendance,57.0,57.0,57.0
1,A15NLokfXbV,2018-01-01,bcg__u1_all,6.0,6.0,6.0
2,A15NLokfXbV,2018-01-01,bcg__u1_static,6.0,6.0,6.0
3,A15NLokfXbV,2018-01-01,dpt1__u1_all,7.0,7.0,7.0
4,A15NLokfXbV,2018-01-01,dpt1__u1_static,7.0,7.0,7.0
...,...,...,...,...,...,...
4597839,zyI8GDUVYdE,2020-08-01,tb_cases_registered,1.0,1.0,1.0
4597840,zyI8GDUVYdE,2020-08-01,td1__pregnant,147.0,147.0,147.0
4597841,zyI8GDUVYdE,2020-08-01,td2__pregnant,120.0,120.0,120.0
4597842,zyI8GDUVYdE,2020-08-01,td3__pregnant,5.0,5.0,5.0


In [62]:
reporting = pd.read_csv('./report_data.csv')

In [63]:
reporting = get_tall_from_wide(reporting, "value_rep")

In [64]:
df = pd.merge(reporting, df, how='left')

In [65]:
del reporting

In [69]:
df

,facility_id,date,indicator,value_rep,value_raw,value_std,value_iqr
0,A0KeWapELXA,2018-01-01,1st_anc_visits,no_positive_indic,NaN,NaN,NaN
1,A0KeWapELXA,2018-01-01,4th_anc_visits,no_positive_indic,NaN,NaN,NaN
2,A0KeWapELXA,2018-01-01,anc_initiated_hiv,no_positive_indic,NaN,NaN,NaN
3,A0KeWapELXA,2018-01-01,anc_tested_hiv,no_positive_indic,NaN,NaN,NaN
4,A0KeWapELXA,2018-01-01,anc_tested_hiv_positive,no_positive_indic,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5730335,aVLWD9zrowf,2020-08-01,td4_5__nonpregnant,no_positive_indic,NaN,NaN,NaN
5730336,aVLWD9zrowf,2020-08-01,td4_5__pregnant,no_positive_indic,NaN,NaN,NaN
5730337,aVLWD9zrowf,2020-08-01,tested_hiv,no_positive_indic,NaN,NaN,NaN
5730338,aVLWD9zrowf,2020-08-01,tested_hiv_positive,no_positive_indic,NaN,NaN,NaN


In [70]:
df.to_csv("repo.csv")